In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
sys.path.append(workdir_ + 'myPythonTools/CASutils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import filter_utils as fu

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
OurMonth = '2004-07'
exp='c153_topfix_ne240pg3_FMTHIST_xic_x02'
#exp='fmthist_MM_control_TEM_2'
B = uti.MakeDict4Exp( exp=exp , user='juliob', subd='hist' , hsPat='cam.h1i' , ymdPat=f'{OurMonth}-*' ,verbose=True, open_dataset=True )

#Q = uti.MakeDict4Exp( exp=exp , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='1994-07*' ,verbose=True, open_dataset=True )
#phis=Q.X.PHIS.values/9.8

In [ ]:
C = uti.MakeDict4Exp( exp=exp , user='juliob', subd='hist' , hsPat='cam.h2i' , ymdPat=f'{OurMonth}-*' ,verbose=True, open_dataset=True )

In [ ]:
######################################
# Subsample C in time if necessary
######################################
if (C.X.dims['time'] > B.X.dims['time']):
    C.X = C.X.sel( time=B.X.time )

In [ ]:

pdata2=C.X.SHFLX.values
ps=C.X.PS.values

"""
lon0,lat0=100.,-50.
zoog = np.where( (np.abs(lat_x - lat0)<1)&(np.abs(lon_x - lon0)<1) )
print(zoog[0])
"""

In [ ]:
########################
# Some useful vars
########################

hyai_x=B.X.hyai.values
hybi_x=B.X.hybi.values
hyam_x=B.X.hyam.values
hybm_x=B.X.hybm.values

#ps_x =B.X.PS.values
lon_x=B.X.lon.values
lat_x=B.X.lat.values
lev_x=B.X.lev.values
zlev_x=-7.0*np.log( lev_x/1_000. )


In [ ]:

kpl=10
print(zlev_x[kpl])
kpl2=80
print(zlev_x[kpl2])
kpl4=68
print(zlev_x[kpl4])


In [ ]:
pdata=B.X.OMEGA[:, kpl, :].values


In [ ]:

print(np.shape(pdata))
print(np.shape(pdata2))


In [ ]:
omega_gridKey = GrU.gridKey( B.X.OMEGA )
print( omega_gridKey )
print( np.shape( pdata ))

In [ ]:
# Only possible on lat-lon field

if (omega_gridKey=='tzyx' ):
    nt,nz,ny,nx = np.shape( B.X.OMEGA )
    ss=np.zeros((nt,ny,nx) )
    for t in np.arange( nt ):
        if ( (t%10)==0):
            print( f'time {t}' ) 
        ss[t,:,:]=fu.filterk( B.X.OMEGA[t,kpl,:,:] , kmin=0,kmax=20,dimlon=1 )
    pdata = pdata - ss



In [ ]:
###


u1km=B.X.U[:, kpl2, :, : ].values
v1km=B.X.V[:, kpl2, :, : ].values

u5km=B.X.U[:, kpl4, :, : ].values
v5km=B.X.V[:, kpl4, :, : ].values



In [ ]:
%%time
zeta1km=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    zeta1km [t,:,:] = nuti.Sphere_Curl2( u1km[t,:,:] , v1km[t,:,:] , lat_x, lon_x, wrap=True )

In [ ]:
%%time
zeta5km=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    zeta5km [t,:,:] = nuti.Sphere_Curl2( u5km[t,:,:] , v5km[t,:,:] , lat_x, lon_x, wrap=True )

In [ ]:

fld1='Omg_44km'
levels1=np.linspace(-.1, .1, num=21) / 2.
cmap1='bwr'

#PS ....
levels2=[98_000., 100_000.] # np.linspace( -100,100,num=11)

fld2='zeta_5km'
fld2='SHFLX'

if ( fld2 == 'SHFLX'):
    pdata2=C.X.SHFLX.values
    levels3 = np.linspace( 0, 250., num=6) #SHFLX levs
    cmap3='magma'

if (fld2 == 'Omg_1km'):
    pdata2=B.X.OMEGA[:, kpl2, :, : ].values
    levels3 = 2*np.linspace(-.1,.1,num=6) 
    cmap3='bwr'

if (fld2 == 'zeta_1km' ):
    pdata2 = zeta1km
    levels3 = 0.2*np.linspace(-.001,.001,num=6)  
    cmap3='PiYG_r'
    
if (fld2 == 'zeta_5km' ):
    pdata2 = zeta5km
    levels3 = 0.2*np.linspace(-.001,.001,num=6)  
    cmap3='PiYG_r'


In [ ]:
#doodoo
print( f"{B.X.time[0].dt.strftime('%Y-%m-%d %HZ').values}" )#.values )
print( len(B.X.time) )

In [ ]:
plt.plot( zeta1km[0,:,:].flatten() )

In [ ]:
nframes = len(B.X.time) 

# Create the figure and axis

MapProj =ccrs.Orthographic(0, -90.)
DataProj = ccrs.PlateCarree()
# Get the name of the projection
proj_name = MapProj.__class__.__name__

#fig, ax = plt.subplots(figsize=(10,10) , projection=MapProj )
fig = plt.figure(figsize=( 20, 10 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes )  #, projection=MapProj)


frame=64

contour = ax.tricontourf(lon_x, lat_x, pdata[frame, :], levels=levels1, cmap=cmap1,alpha=0.5 )  #,transform=DataProj )
contour3 = ax.tricontourf(lon_x, lat_x, pdata2[frame, :], levels=levels3 , cmap=cmap3 , extend='both', alpha=.25 )  #,transform=DataProj ) #OMEGA 1Km
contour2 = ax.tricontour(lon_x, lat_x, ps[frame,:], levels=levels2 , alpha=0.5, colors='black' )  # Add a second contour plot for data2
annot= ax.text( 0.05,.05,B.X.time[frame].dt.strftime('%Y-%m-%d %HZ').values , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
annot2= ax.text( 0.5,.99, f'Z={zlev_x[kpl]:.0f} km' , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
cbar = plt.colorbar(contour3, ax=ax, shrink=0.6)  # Create colorbar



In [ ]:
nframes = len(B.X.time) 
#levels=0.2*np.linspace(-.2, .2, num=21)
#levels=np.linspace(-.2, .2, num=21)
#levels2=np.linspace( 240,300,num=11)
levels=np.linspace(-.1, .1, num=21) / 2.
levels2=[98_000., 100_000.] # np.linspace( -100,100,num=11)
levels3 = np.linspace( 0, 250., num=6) #SHFLX levs
levels3 = 21 # 2*np.linspace(-.1,.1,num=6)  #OMEGA 1km levs
# Create the figure and axis

MapProj =ccrs.Orthographic(0, -90.)
DataProj = ccrs.PlateCarree()
# Get the name of the projection
proj_name = MapProj.__class__.__name__

#fig, ax = plt.subplots(figsize=(10,10) , projection=MapProj )
fig = plt.figure(figsize=( 20, 10 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes )  #, projection=MapProj)


frame=0

#contour = ax.contourf(lon_x, lat_x, pdata[frame, :,:], levels=levels, cmap='bwr',alpha=0.5 )  #,transform=DataProj )
#contour3 = ax.contourf(lon_x, lat_x, pdata2[frame, :,:], levels=levels3 , cmap='magma' , alpha=.25 )  #,transform=DataProj )
contour3 = ax.contourf(lon_x, lat_x, pdata2[frame, :,:], levels=levels3 , cmap='bwr' , alpha=.25 )  #,transform=DataProj ) #OMEGA 1Km
contour2 = ax.contour(lon_x, lat_x, ps[frame,:,:], levels=levels2 , alpha=0.5, colors='black' )  # Add a second contour plot for data2
annot= ax.text( 0.05,.05,B.X.time[frame].dt.strftime('%Y-%m-%d %HZ').values , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
annot2= ax.text( 0.5,.99, f'Z={zlev_x[kpl]:.0f} km' , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar



In [ ]:
print( f'this is where stop')
levels2 = [ 80000., 96000., 98000.0, 100000.0]
print(levels2)

In [ ]:
nframes = len(B.X.time) 
#levels=0.2*np.linspace(-.2, .2, num=21)
#levels=np.linspace(-.2, .2, num=21)
#levels2=np.linspace( 240,300,num=11)

# Create the figure and axis


#fig, ax = plt.subplots(figsize=(10,10) , projection=MapProj )
fig = plt.figure(figsize=( 20, 10 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes )  #, projection=MapProj)


frame=0

contour = ax.contourf(lon_x, lat_x, pdata[frame, :,:], levels=levels1, cmap=cmap1 , alpha=0.5 )  #,transform=DataProj )
contour3 = ax.contourf(lon_x, lat_x, pdata2[frame, :,:], levels=levels3 , cmap=cmap3 , alpha=.25 )  #,transform=DataProj )
contour2 = ax.contour(lon_x, lat_x, ps[frame,:,:], levels=levels2 , alpha=0.5, colors='black' )  # Add a second contour plot for data2
ax.tick_params(axis='both', which='major',direction='in',labelsize=10,pad=-20)


annot= ax.text( 0.05,.05,B.X.time[frame].dt.strftime('%Y-%m-%d %HZ').values , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
annot2= ax.text( 0.5,.99, f'Z={zlev_x[kpl]:.0f} km' , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar



In [ ]:
# NO CARTOPY ===================================================

from matplotlib.animation import FuncAnimation
from IPython.display import HTML
# Set the embed limit to a larger value (e.g., 50 MB)
plt.rcParams['animation.embed_limit'] = 50  # You can adjust this value as needed

nframes = len(B.X.time) 
#levels=0.2*np.linspace(-.2, .2, num=21)
#levels=np.linspace(-.2, .2, num=21)
#levels2=np.linspace( 240,300,num=11)

# Create the figure and axis

MapProj =ccrs.Orthographic(0, -90.)
DataProj = ccrs.PlateCarree()
# Get the name of the projection
proj_name = MapProj.__class__.__name__

#fig, ax = plt.subplots(figsize=(10,10) , projection=MapProj )
fig = plt.figure(figsize=( 20, 10 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes )  #, projection=MapProj)


frame=0

contour = ax.contourf(lon_x, lat_x, pdata[frame, :,:], levels=levels1, cmap=cmap1 , alpha=0.5 )  #,transform=DataProj )
contour3 = ax.contourf(lon_x, lat_x, pdata2[frame, :,:], levels=levels3 , cmap=cmap3 , alpha=.25 )  #,transform=DataProj )
contour2 = ax.contour(lon_x, lat_x, ps[frame,:,:], levels=levels2 , alpha=0.5, colors='black' )  # Add a second contour plot for data2
ax.tick_params(axis='both', which='major',direction='in',labelsize=10,pad=-20)


annot= ax.text( 0.05,.05,B.X.time[frame].dt.strftime('%Y-%m-%d %HZ').values , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
annot2= ax.text( 0.5,.99, f'Z={zlev_x[kpl]:.0f} km' , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar


# Update function for the animation
def update(frame):
    global contour ,contour2,contour3 , ax
    for coll in contour.collections:
        coll.remove()  # Remove existing contour collections
    for coll in contour2.collections:
        coll.remove()  # Remove existing contour collections
    for coll in contour3.collections:
        coll.remove()  # Remove existing contour collections
    contour = ax.contourf(lon_x, lat_x, pdata[frame, :,:], levels=levels1, cmap=cmap1 , alpha=0.5 )  #,transform=DataProj )
    contour3 = ax.contourf(lon_x, lat_x, pdata2[frame, :,:], levels=levels3 , cmap=cmap3 , alpha=.25 )  #,transform=DataProj )
    contour2 = ax.contour(lon_x, lat_x, ps[frame,:,:], levels=levels2 , alpha=0.5,colors='black')  # Add a second contour plot for data2
    ax.tick_params(axis='both', which='major',direction='in',labelsize=10,pad=-20)

    annot= ax.text( 0.05,.05,B.X.time[frame].dt.strftime('%Y-%m-%d %HZ').values , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
    annot2= ax.text( 0.5,.99, f'Z={zlev_x[kpl]:.0f} km' , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
    annot3= ax.text( 0.25,.05,f'fld1: {fld1}, fld2: {fld2}' , transform=ax.transAxes, fontsize=10, bbox=dict(facecolor='white') )
   #cbar.update_normal(contour)  # Update colorbar
    #cbar = plt.colorbar(contour, shrink=.6)
    return contour.collections +contour2.collections+contour3.collections

# Create and display the animation
#nframes=2
ani = FuncAnimation(fig, update, frames=nframes, interval=50, blit=True)
plt.close()  # Prevents the initial empty plot from showing up
HTML(ani.to_html5_video())



In [ ]:
#aniName = f'../AnimationFiles/{exp}-Omg44km-PS-SHFLX-{OurMonth}.mp4'
aniName = f'../AnimationFiles/{exp}-Omg_{zlev_x[kpl]:.0f}km-PS-SHFLX-{OurMonth}.mp4'
aniName = f'../AnimationFiles/{exp}-Omg_{zlev_x[kpl]:.0f}km-PS-{fld2}-{OurMonth}.mp4'

print(aniName)

In [ ]:
# Step 7: Save the animation as HTML5 video
ani.save( aniName , writer='ffmpeg')

In [ ]:

# Create the figure and axis

MapProj =ccrs.Orthographic(0, -90.)
DataProj = ccrs.PlateCarree()
# Get the name of the projection
proj_name = MapProj.__class__.__name__

#fig, ax = plt.subplots(figsize=(10,10) , projection=MapProj )
fig = plt.figure(figsize=( 20, 10 ))
Axes = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax = fig.add_axes( Axes )  #, projection=MapProj)


frame=62

contour = ax.contourf(lon_x, lat_x, pdata[frame, :,:], levels=levels, cmap=cmap1 , alpha=0.5 )  #,transform=DataProj )
contour3 = ax.contourf(lon_x, lat_x, pdata2[frame, :,:], levels=levels3 , cmap=cmap3 , alpha=.25 )  #,transform=DataProj )
contour2 = ax.contour(lon_x, lat_x, ps[frame,:,:], levels=levels2 , alpha=0.5, colors='black' )  # Add a second contour plot for data2
annot= ax.text( 0.05,.05,B.X.time[frame].dt.strftime('%Y-%m-%d %HZ').values , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
annot2= ax.text( 0.5,.99, f'Z={zlev_x[kpl]:.0f} km' , transform=ax.transAxes, fontsize=16, bbox=dict(facecolor='white') )
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar



In [ ]:
shflx=C.X.SHFLX.values

og1km=B.X.OMEGA[:, kpl2, :, : ].values



In [ ]:
j0,j1=250,350
print( lat_x[[j0,j1]] )

poop=np.average( (pdata[:,j0:j1,:])**2,axis=1)
plt.contour(poop,colors='black',levels=21 )
poop2=np.average( (zeta1km[:,j0:j1,:])**2,axis=1)
#plt.contour(poop2,colors='red',levels=21 )
poop3=np.average( (og1km[:,j0:j1,:])**2,axis=1)
plt.contour(poop3,colors='green',levels=21 )
poop4=np.average( (shflx[:,j0:j1,:])**2,axis=1)
#plt.contour(poop4,colors='orange')


In [ ]:
ntime,ilat =64,340
ntime,ilat =64,350
ntime,ilat =64,380
print(B.X.date[ntime])
og=B.X.OMEGA[ntime,:,:,:].values

In [ ]:
print(np.shape(og [:,ilat,:].T/ lev_x))
#wog=(og [:,ilat,:].T / lev_x).T
wog=og [:,ilat,:]

In [ ]:
print(lat_x[ilat])
plt.contourf(lon_x,zlev_x,wog,cmap='bwr',levels=2*np.linspace(-.1,.1,21) )
plt.xlim(180,210)
plt.ylim(0,50)
plt.colorbar()


In [ ]:
#HTML(ani.to_html5_video())

In [ ]:
import math
result = math.ceil(22 / 3)
print(result)  # 